
### Data Reading

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [0]:
df = spark.read.format("parquet")\
    .load("s3://argha-associatede-s3/bronze/orders/")

df.display()

In [0]:
df.printSchema()

In [0]:
df = df .withColumn("order_date", to_timestamp("order_date"))

df.display()

In [0]:
df = df.withColumn("year", year("order_date"))

df.display()

In [0]:
df1 = df.withColumn("sales_dense_rank_by_year",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

df1.display()

In [0]:
df2 = df1.withColumn("sales_rank_by_year",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

df2.display()


## Using Python class to make the code less repetitive

In [0]:
class Windows:
  def __init__(self,df):
    self.df = df 

  def dense_rank(self):
      df_dense_rank = self.df.withColumn("dense_rank",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
      return df_dense_rank
  
  def rank(self):
      df_rank = self.df.withColumn("rank",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    
  def row_num(self):
      df_row_num = self.df.withColumn("row_num",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
      return df_row_num


In [0]:
df_new = df

df_new.display()

In [0]:
obj = Windows(df_new)

df_result = obj.dense_rank()    

df_result.display()


### Data Writing

In [0]:
df.write.format("delta").mode("overwrite").save("s3://argha-associatede-s3/silver/orders/")